In [0]:
import numpy as np 
import sklearn.metrics
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import torch 
from tqdm import tqdm 
import os 
import torchvision

In [0]:
model = models.resnet18(pretrained = True)
model.conv1 = torch.nn.Conv2d(1,64,kernel_size = 7, stride =2, padding=3,bias=False)
model.fc = torch.nn.Linear(model.fc.in_features,10)

In [5]:
mnist = torchvision.datasets.MNIST('./var',download = True)
transform = transforms.Compose([transforms.ToTensor()])
train = torchvision.datasets.MNIST('./var',train =True, transform = transform)
trainloader = torch.utils.data.DataLoader(train,batch_size =32,shuffle=True)
test = torchvision.datasets.MNIST('./var',train =False, transform = transform)
testloader = torch.utils.data.DataLoader(test,batch_size =len(test),shuffle=True)

Extracting ./var/MNIST/raw/train-images-idx3-ubyte.gz to ./var/MNIST/raw



Extracting ./var/MNIST/raw/train-labels-idx1-ubyte.gz to ./var/MNIST/raw


Extracting ./var/MNIST/raw/t10k-images-idx3-ubyte.gz to ./var/MNIST/raw


Extracting ./var/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./var/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [10]:

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model.to(device)

for epoch in range(20):
    totoal_loss = 0
    for inputs,outputs in trainloader:
        inputs = inputs.to(device)
        outputs = outputs.to(device)
        optimizer.zero_grad()
        results = model(inputs)
        loss = loss_function(results,outputs)
        totoal_loss += loss.item()
        loss.backward()
        optimizer.step()
    print ('Loss:{0}'.format(totoal_loss/len(trainloader)))



Loss:0.20647160531083744
Loss:0.08286114345391592
Loss:0.06405865077575047
Loss:0.05047977794210116
Loss:0.049372895564635595
Loss:0.03290026944975058
Loss:0.02729987442791462
Loss:0.031353699031472206
Loss:0.024639270094037057
Loss:0.017411595424016317
Loss:0.018734765725334484
Loss:0.016761559263865154
Loss:0.012414484297235807
Loss:0.011763108477493128
Loss:0.013081124787032604
Loss:0.015286233340452114
Loss:0.0096629155429701
Loss:0.008245539770027002
Loss:0.00807846652319034
Loss:0.009926744587471089


NameError: ignored

In [11]:
import sklearn.metrics
for inputs,actual in testloader:
    inputs = inputs.to(device)
    results = model(inputs).argmax(dim=1).to('cpu').numpy()
    accuracy = sklearn.metrics.accuracy_score(actual,results)
    print(accuracy)
print(sklearn.metrics.classification_report(actual,results))

0.9939
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      1.00      1135
           2       1.00      0.99      1.00      1032
           3       0.99      1.00      0.99      1010
           4       1.00      0.99      1.00       982
           5       1.00      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       1.00      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

